# Data Engineering Project: Database Design & REST API Implementation
By: Cem Bayramoglu, Semen Petrov, Annette Martin

## Project Overview
Design and implement a relational database system with REST API access following a three-step workflow: data modeling, database implementation, and API interaction.

### **Music Streaming** - Songs, artists, albums, users, playlists, subscriptions

In [1]:
# @title Code to get information about top songs and such
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_kworb_table(url, table_name):
    """
    Scrape a table from kworb.net

    Args:
        url: The URL to scrape
        table_name: A descriptive name for this table

    Returns:
        DataFrame with the scraped data
    """
    try:
        print(f"Scraping {table_name} from {url}...")

        # Fetch webpage
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the table
        table = soup.find('table')

        if not table:
            print(f"  ⚠️ No table found at {url}")
            return None

        # Extract headers
        headers = []
        for th in table.find_all('th'):
            headers.append(th.text.strip())

        # Extract rows
        rows = []
        for tr in table.find_all('tr')[1:]:  # Skip header row
            cells = tr.find_all('td')
            row = [cell.text.strip() for cell in cells]
            if row:
                rows.append(row)

        # Create DataFrame
        df = pd.DataFrame(rows, columns=headers)
        print(f"  ✓ Successfully scraped {len(df)} rows")

        return df

    except Exception as e:
        print(f"  X Error scraping {table_name}: {e}")
        return None


# Define your URLs and their names
urls_to_scrape = {
    'Apple_Music_World_Wide_Songs': 'https://kworb.net/apple_songs/',
    'Billboard_Radio_Songs': 'https://kworb.net/radio/',
    'Itunes_World_Wide_Songs': 'https://kworb.net/ww/',
    'Spotify_Most_Streamed_Artists_of_All_Time': 'https://kworb.net/spotify/artists.html',
    'Spotify_Most_Streamed_Songs': 'https://kworb.net/spotify/toplists.html',
    'Spotify_Top_Artists_by_Monthly_Listeners': 'https://kworb.net/spotify/listeners.html',
    'YouTube_Archive': 'https://kworb.net/youtube/archive.html',
    'YouTube_Most_Viewed_Artists': 'https://kworb.net/youtube/archive.html',
}

# Dictionary to store all DataFrames
dataframes = {}

# Scrape each URL
for name, url in urls_to_scrape.items():
    df = scrape_kworb_table(url, name)
    if df is not None:
        dataframes[name] = df

    # Be polite - wait between requests
    time.sleep(1)

print("SCRAPING COMPLETE")

Scraping Apple_Music_World_Wide_Songs from https://kworb.net/apple_songs/...
  ✓ Successfully scraped 200 rows
Scraping Billboard_Radio_Songs from https://kworb.net/radio/...
  ✓ Successfully scraped 167 rows
Scraping Itunes_World_Wide_Songs from https://kworb.net/ww/...
  ✓ Successfully scraped 200 rows
Scraping Spotify_Most_Streamed_Artists_of_All_Time from https://kworb.net/spotify/artists.html...
  ✓ Successfully scraped 3000 rows
Scraping Spotify_Most_Streamed_Songs from https://kworb.net/spotify/toplists.html...
  ✓ Successfully scraped 24 rows
Scraping Spotify_Top_Artists_by_Monthly_Listeners from https://kworb.net/spotify/listeners.html...
  ✓ Successfully scraped 2500 rows
Scraping YouTube_Archive from https://kworb.net/youtube/archive.html...
  ✓ Successfully scraped 1730 rows
Scraping YouTube_Most_Viewed_Artists from https://kworb.net/youtube/archive.html...
  ✓ Successfully scraped 1730 rows
SCRAPING COMPLETE


In [2]:
# @title DataFrames
# Display info about each DataFrame
for name, df in dataframes.items():
    print(f"\n{name}:")
    print(f"  Rows: {len(df)}")
    print(f"  Columns: {list(df.columns)}")

# Access individual DataFrames
df_Spotify_Listeners = dataframes.get('Spotify_Listeners')
df_YouTube_Archive = dataframes.get('YouTube_Archive')
df_Pop_Charts = dataframes.get('Pop_Charts')

# Display first DataFrame as example
if df_Spotify_Listeners is not None:
    print("\n\nSpotify Listeners Preview:")
    display(df_Spotify_Listeners.head())


Apple_Music_World_Wide_Songs:
  Rows: 200
  Columns: ['Pos', 'P+', 'Artist and Title', 'Days', 'Pk', '(x?)', 'Pts', 'Pts+', 'TPts', 'US', 'UK', 'JP', 'DE']

Billboard_Radio_Songs:
  Rows: 167
  Columns: ['Pos', 'P+', 'Artist and Title', 'Days', 'Pk', '(x?)', 'Aud', 'Aud+', 'Formats', 'PkAud', 'iTunes', 'Spotify', 'Apple M', 'Shazam']

Itunes_World_Wide_Songs:
  Rows: 200
  Columns: ['Pos', 'P+', 'Artist and Title', 'Days', 'Pk', '(x?)', 'Pts', 'Pts+', 'TPts', 'US', 'UK', 'DE', 'AU', 'JP']

Spotify_Most_Streamed_Artists_of_All_Time:
  Rows: 3000
  Columns: ['Artist', 'Streams', 'Daily', 'As lead', 'Solo', 'As feature']

Spotify_Most_Streamed_Songs:
  Rows: 24
  Columns: ['', 'Artist and Title', 'Streams']

Spotify_Top_Artists_by_Monthly_Listeners:
  Rows: 2500
  Columns: ['#', 'Artist', 'Listeners', 'Daily +/-', 'Peak', 'PkListeners']

YouTube_Archive:
  Rows: 1730
  Columns: ['Artist', 'Total', '100M']

YouTube_Most_Viewed_Artists:
  Rows: 1730
  Columns: ['Artist', 'Total', '100M']


In [3]:
# @title Saving
for name, df in dataframes.items():
    df.to_csv(f'{name}.csv', index=False)
    print(f"Saved {name}.csv")

# Find the file by clicking on the file icon in the left toolbar
  # Then download from there

Saved Apple_Music_World_Wide_Songs.csv
Saved Billboard_Radio_Songs.csv
Saved Itunes_World_Wide_Songs.csv
Saved Spotify_Most_Streamed_Artists_of_All_Time.csv
Saved Spotify_Most_Streamed_Songs.csv
Saved Spotify_Top_Artists_by_Monthly_Listeners.csv
Saved YouTube_Archive.csv
Saved YouTube_Most_Viewed_Artists.csv


In [4]:
# @title Code for music metadata
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from typing import Dict, List, Optional

class MusicBrainzMetadata:
    """
    Comprehensive MusicBrainz API client
    Rate limit: 2 seconds between requests
    """

    def __init__(self, app_name: str = "MusicMetadataCollector", app_version: str = "1.0", contact: str = "annette.hatch.martin@gmail.com"):
        self.base_url = "https://musicbrainz.org/ws/2"
        self.coverart_url = "https://coverartarchive.org"
        self.headers = {
            'User-Agent': f'{app_name}/{app_version} ( {contact} )'
        }
        self.rate_limit_delay = 2.0

    def get_comprehensive_artist_metadata(self, artist_id: str) -> Dict:
        """Get ALL available metadata for an artist"""
        endpoint = f"{self.base_url}/artist/{artist_id}"
        params = {
            'fmt': 'json',
            'inc': 'recordings+releases+release-groups+works+url-rels+artist-rels+tags+ratings+genres+annotation+aliases'
        }

        response = requests.get(endpoint, headers=self.headers, params=params)
        response.raise_for_status()
        time.sleep(self.rate_limit_delay)
        return response.json()

    def search_and_get_full_artist_data(self, artist_name: str) -> Dict:
        """Search for an artist and return ALL metadata"""
        # First search for artist
        endpoint = f"{self.base_url}/artist"
        params = {
            'query': f'artist:"{artist_name}"',
            'fmt': 'json',
            'limit': 1
        }

        print(f"  Searching: {artist_name}...", end=" ")
        response = requests.get(endpoint, headers=self.headers, params=params)
        response.raise_for_status()
        data = response.json()

        if not data.get('artists'):
            print(f"❌ Not found")
            return None

        artist_id = data['artists'][0]['id']
        time.sleep(self.rate_limit_delay)

        # Get detailed artist info
        print(f"✓ Getting metadata...", end=" ")
        artist_data = self.get_comprehensive_artist_metadata(artist_id)

        # Extract URL relationships
        url_relations = artist_data.get('relations', [])
        urls = {}
        for rel in url_relations:
            rel_type = rel.get('type', '')
            url = rel.get('url', {}).get('resource', '')
            if url:
                urls[rel_type] = url

        # Extract release groups
        release_groups = artist_data.get('release-groups', [])
        albums = [rg for rg in release_groups if rg.get('primary-type') == 'Album']
        singles = [rg for rg in release_groups if rg.get('primary-type') == 'Single']
        eps = [rg for rg in release_groups if rg.get('primary-type') == 'EP']

        # Extract aliases
        aliases = [alias.get('name') for alias in artist_data.get('aliases', [])]

        print("✓ Complete")

        return {
            # Identifiers
            'artist_mbid': artist_data.get('id'),
            'isni': ', '.join(artist_data.get('isnis', [])),
            'ipi': ', '.join(artist_data.get('ipis', [])),

            # Basic Info
            'name': artist_data.get('name'),
            'sort_name': artist_data.get('sort-name'),
            'type': artist_data.get('type'),
            'country': artist_data.get('country'),
            'gender': artist_data.get('gender'),
            'disambiguation': artist_data.get('disambiguation', ''),
            'aliases': ', '.join(aliases[:5]) if aliases else '',

            # Dates
            'begin_date': artist_data.get('life-span', {}).get('begin', ''),
            'end_date': artist_data.get('life-span', {}).get('end', ''),
            'is_ended': artist_data.get('life-span', {}).get('ended', False),

            # Statistics
            'num_albums': len(albums),
            'num_singles': len(singles),
            'num_eps': len(eps),
            'total_releases': len(artist_data.get('releases', [])),
            'total_recordings': len(artist_data.get('recordings', [])),

            # Categorization
            'genres': ', '.join([g['name'] for g in artist_data.get('genres', [])]),
            'tags': ', '.join([t['name'] for t in artist_data.get('tags', [])[:10]]),

            # Community Data
            'rating': artist_data.get('rating', {}).get('value'),
            'rating_votes': artist_data.get('rating', {}).get('votes-count', 0),

            # Official Links
            'official_website': urls.get('official homepage', ''),
            'wikipedia': urls.get('wikipedia', ''),
            'wikidata': urls.get('wikidata', ''),
            'discogs': urls.get('discogs', ''),
            'allmusic': urls.get('allmusic', ''),
            'imdb': urls.get('imdb', ''),
            'bandcamp': urls.get('bandcamp', ''),

            # Streaming Platforms
            'spotify': urls.get('spotify', ''),
            'apple_music': urls.get('apple music', ''),
            'youtube': urls.get('youtube', ''),
            'youtube_music': urls.get('youtube music', ''),
            'soundcloud': urls.get('soundcloud', ''),
            'deezer': urls.get('deezer', ''),
            'tidal': urls.get('tidal', ''),
            'amazon_music': urls.get('amazon music', ''),

            # Social Media
            'twitter': urls.get('twitter', ''),
            'instagram': urls.get('instagram', ''),
            'facebook': urls.get('facebook', ''),
            'tiktok': urls.get('tiktok', ''),
            'linkedin': urls.get('linkedin', ''),

            # Other
            'lyrics': urls.get('lyrics', ''),
            'purchase_url': urls.get('purchase for mail-order', ''),
            'crowdfunding': urls.get('crowdfunding', ''),
        }

    def get_multiple_artists_metadata(self, artist_names: List[str]) -> pd.DataFrame:
        """Get comprehensive metadata for multiple artists"""
        all_artists_data = []

        print(f"\n{'='*70}")
        print(f"Fetching metadata for {len(artist_names)} artists")
        print(f"Rate limit: {self.rate_limit_delay} seconds between requests")
        print(f"Estimated time: {len(artist_names) * self.rate_limit_delay * 2 / 60:.1f} minutes")
        print(f"{'='*70}\n")

        for idx, artist_name in enumerate(artist_names, 1):
            print(f"[{idx}/{len(artist_names)}]", end=" ")

            try:
                artist_data = self.search_and_get_full_artist_data(artist_name)

                if artist_data:
                    all_artists_data.append(artist_data)
                else:
                    all_artists_data.append({'name': artist_name})

            except Exception as e:
                print(f"  ❌ Error: {e}")
                all_artists_data.append({'name': artist_name, 'error': str(e)})

        print(f"\n{'='*70}")
        print(f"✅ Completed: {len(all_artists_data)} artists processed")
        print(f"{'='*70}\n")

        return pd.DataFrame(all_artists_data)


def scrape_kworb_top_artists(url: str, limit: int = 2000) -> List[str]:
    """
    Scrape top artists from kworb.net

    Args:
        url: kworb.net URL (e.g., 'https://kworb.net/spotify/listeners.html')
        limit: Maximum number of artists to retrieve (default 2000)

    Returns:
        List of artist names
    """
    print(f"\n{'='*70}")
    print(f"Scraping top {limit} artists from kworb.net")
    print(f"{'='*70}\n")

    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table')
    if not table:
        raise Exception("No table found on page")

    artists = []
    rows = table.find_all('tr')[1:]  # Skip header

    for idx, tr in enumerate(rows[:limit], 1):
        cells = tr.find_all('td')
        if len(cells) >= 2:  # Make sure there are enough columns
            # Usually artist is in the second column (index 1)
            artist_name = cells[1].text.strip()
            if artist_name:
                artists.append(artist_name)
                if idx % 100 == 0:
                    print(f"  Scraped {idx} artists...")

    print(f"\n✓ Successfully scraped {len(artists)} artists from kworb.net\n")
    return artists


# ============================================================================
# Main Execution
# ============================================================================
print("="*70)
print("AUTOMATIC TOP ARTISTS METADATA COLLECTOR")
print("="*70)
print(f"Email: annette.hatch.martin@gmail.com")
print(f"Rate Limit: 2 seconds between requests")
print("="*70)

# Initialize API client
mb = MusicBrainzMetadata(
    app_name="AutoTopArtistsCollector",
    app_version="1.0",
    contact="annette.hatch.martin@gmail.com"
)

# Step 1: Scrape top artists from kworb.net
kworb_url = "https://kworb.net/spotify/listeners.html"
num_artists = 2000  # Start with 100 for testing - change to 2000 for full run

print("\n⚠️  IMPORTANT: Starting with 100 artists for testing")
print("⚠️  Change 'num_artists = 2000' in the code for the full dataset")
print("⚠️  Full run (2000 artists) will take ~2 hours with 2-second rate limit\n")

top_artists = scrape_kworb_top_artists(kworb_url, limit=num_artists)

# Step 2: Get MusicBrainz metadata for all artists
df_artists = mb.get_multiple_artists_metadata(top_artists)

# Display results
print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)
print(f"Total artists processed: {len(df_artists)}")
print(f"Successful matches: {df_artists['artist_mbid'].notna().sum()}")
print(f"Failed matches: {df_artists['artist_mbid'].isna().sum()}")
print(f"\nDataFrame shape: {df_artists.shape[0]} rows × {df_artists.shape[1]} columns")

print("\n📊 COLUMN HEADERS (Metadata Fields):")
for col in df_artists.columns:
    print(f"  • {col}")

print("\n\nPreview of first 5 artists:")
display(df_artists.head())

print("\n\n💾 SAVE OPTIONS:")
print("  df_artists.to_csv('top_artists_metadata.csv', index=False)")
print("  df_artists.to_json('top_artists_metadata.json', orient='records')")
print("  df_artists.to_excel('top_artists_metadata.xlsx', index=False)")

print("\n✅ Ready! DataFrame stored in variable: df_artists")

AUTOMATIC TOP ARTISTS METADATA COLLECTOR
Email: annette.hatch.martin@gmail.com
Rate Limit: 2 seconds between requests

⚠️  IMPORTANT: Starting with 100 artists for testing
⚠️  Change 'num_artists = 2000' in the code for the full dataset
⚠️  Full run (2000 artists) will take ~2 hours with 2-second rate limit


Scraping top 2000 artists from kworb.net

  Scraped 100 artists...
  Scraped 200 artists...
  Scraped 300 artists...
  Scraped 400 artists...
  Scraped 500 artists...
  Scraped 600 artists...
  Scraped 700 artists...
  Scraped 800 artists...
  Scraped 900 artists...
  Scraped 1000 artists...
  Scraped 1100 artists...
  Scraped 1200 artists...
  Scraped 1300 artists...
  Scraped 1400 artists...
  Scraped 1500 artists...
  Scraped 1600 artists...
  Scraped 1700 artists...
  Scraped 1800 artists...
  Scraped 1900 artists...
  Scraped 2000 artists...

✓ Successfully scraped 2000 artists from kworb.net


Fetching metadata for 2000 artists
Rate limit: 2.0 seconds between requests
Estima

,artist_mbid,isni,ipi,name,sort_name,type,country,gender,disambiguation,aliases,...,amazon_music,twitter,instagram,facebook,tiktok,linkedin,lyrics,purchase_url,crowdfunding,error
0,afb680f2-b6eb-4cd7-a70b-a63b25c763d5,0000000117226071,"00477487104, 00583145346, 00583145444",Bruno Mars,"Mars, Bruno",Person,US,Male,US singer,"Bruno Mars, Bruno Mars, Peter Gene Hernandez, ...",...,,,,,,,https://www.musixmatch.com/artist/Bruno-Mars,,,NaN
1,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,0000000382556637,00574514146,The Weeknd,"Weeknd, The",Person,CA,Male,Canadian R&B singer,"Abel Makkonen Tesfaye, Abel Tesfaye, The Weeke...",...,,,,,,,https://www.musixmatch.com/artist/The-Weeknd,https://www.cdjapan.co.jp/person/700613925,,NaN
2,e0140a67-e4d1-4f13-8a01-364355bee46e,0000000114791200,"00611935265, 00611935363",Justin Bieber,"Bieber, Justin",Person,CA,Male,,"Justin Beiber, Justin Bieber, Justin Drew Bieb...",...,,,,,,,https://www.musixmatch.com/artist/Justin-Bieber,,,NaN
3,f4fdbb4c-e4b7-47a0-b83b-d91bbfcfa387,0000000059110272,00664244638,Ariana Grande,"Grande, Ariana",Person,US,Female,,"Ariana Grande-Butera, אריאנא גראנדע, آریانا گر...",...,,,,,,,http://www.directlyrics.com/ariana-grande-arti...,,,NaN
4,20244d07-534f-4eff-b4d4-930878889970,0000000078519858,"00454808047, 00454808145, 00454808243, 0080435...",Taylor Swift,"Swift, Taylor",Person,US,Female,,"Dr. Taylor Alison Swift, Nils Sjöberg, T-Swizz...",...,,,,,,,https://www.musixmatch.com/artist/Taylor-Swift,https://www.yesasia.com/0-aid1895644-0-bpt.47-...,,NaN




💾 SAVE OPTIONS:
  df_artists.to_csv('top_artists_metadata.csv', index=False)
  df_artists.to_json('top_artists_metadata.json', orient='records')
  df_artists.to_excel('top_artists_metadata.xlsx', index=False)

✅ Ready! DataFrame stored in variable: df_artists


In [5]:
# @title Export to CSV
df_artists.to_csv('top_artists_metadata.csv', index=False)
print("✅ Exported to: top_artists_metadata.csv")

✅ Exported to: top_artists_metadata.csv
